In [4]:
import numpy as np
import pandas as pd
import seaborn as sns


In [11]:
#importing csv file using pandas
df = pd.read_csv("questions.csv")
df.shape

(404351, 6)

In [8]:
#sampling top 10 data sets
df.sample(10)

,id,qid1,qid2,question1,question2,is_duplicate
115939,115939,229826,229827,What are some inventions of the Renaissance re...,What are some inventions of the Renaissance re...,0
248700,248700,489698,489699,How can I grow taller?,I'm 16 and 5'2.5. How can I ACTIVELY grow taller?,1
312102,312102,612608,612609,How to creat a blog on quora?,How can I create a blog on Quora?,1
42394,42394,84435,84436,Why didn't Marvel continue Andrew Garfield as ...,Why didn't Marvel cast Andrew Garfield as Spid...,1
390107,390107,762554,762555,What are the best ways to lose weight?,What is the best way to lose weight and not ga...,1
237500,237500,467854,467855,What is the difference between provident fund ...,What is the difference between a core bond fun...,0
196382,196382,387596,387597,What other business options with computer cent...,Does love drive us to go on living?,0
152724,152724,302084,302085,What is a charity?,What is charity?,1
1028,1028,2057,2058,"Does Stripe or Square-Up act as an ""agent"" to ...",Is a license needed to operate as a payment pr...,0
338179,338179,662966,662967,Which is the best book or material to understa...,What are the best books for control engineerin...,0


In [12]:
#checking if there are missing values
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [10]:
#checking if there is duplicate values(rows)
df.duplicated().sum()

0